In [11]:
import routeros_api2
import time
import sys
import random
import matplotlib
matplotlib.use('Qt5Agg')

from PyQt5 import QtCore, QtWidgets

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure

In [12]:
main_interface=routeros_api2.Api(address='192.168.88.4')

In [13]:
def align():
    routeros_api2.Api.talk(main_interface,'/interface/w60g/monitor\n=numbers="wlan60-2"\n=once')


In [14]:
txsector1=0
def rssi():
    data=[]
    global txsector1
    for i in range(0,1):
        time.sleep(.1)
        recv=(routeros_api2.Api.talk(main_interface,'/interface/w60g/monitor\n=numbers="wlan60-2"\n=once'))
        data.append(recv[0]['rssi'])
        txsector1=recv[0]['signal']
    #align()
    print(recv[0])
    return data
    #time.sleep(.15)
    #r=routeros_api2.Api.talk(main_interface,[('/interface/w60g/set\n=numbers="wlan60-2"\n=tx-sector=10')])
    #print(r)


In [15]:
rssi()

{'connected': 'true', 'frequency': '58320', 'remote-address': 'C4:AD:34:AE:FD:15', 'tx-mode': 'dmg', 'tx-mcs': '8', 'tx-phy-rate': '2310000000', 'signal': '95', 'rssi': '-49', 'tx-sector': '62', 'tx-sector-info': 'left 19 degrees, up 26.6 degrees', 'distance': '2.95', 'tx-packet-error-rate': '9'}


['-49']

In [5]:
def txsector():
    data=[]
    for i in range(0,1):
        time.sleep(10)
        recv=(routeros_api2.Api.talk(main_interface,'/interface/w60g/monitor\n=numbers="wlan60-2"\n=once'))
        data.append(recv[0]['tx-sector'])
    #align()
    return recv
    #time.sleep(.15)
    #r=routeros_api2.Api.talk(main_interface,[('/interface/w60g/set\n=numbers="wlan60-2"\n=tx-sector=10')])
    #print(r)

In [6]:
# def txsector_auto():
#     for i in range(0,60):
#         routeros_api2.Api.talk(main_interface,[('/interface/w60g/set\n=numbers="wlan60-2"\n=tx-sector='+str(i))])
#         #time.sleep(20)
#         rssi()


In [7]:
# for i in range(60,68):
#     a=rssi()
#     print(i,a)
#     print(txsector1)
#     routeros_api2.Api.talk(main_interface,[('/interface/w60g/set\n=numbers="wlan60-2"\n=tx-sector='+str(i))])

In [8]:
rssi()

txsector1

'95'

In [9]:
import dash
from dash.dependencies import Output, Input
from dash import dcc
from dash import html
import plotly
import random
import plotly.graph_objs as go
from collections import deque

X = deque(maxlen = 20)
X.append(1)

Y = deque(maxlen = 20)
Y.append(-65)

X1 = deque(maxlen = 20)
X1.append(1)

Y1 = deque(maxlen = 20)
Y1.append(0)
app = dash.Dash(__name__)

#app.layout = html.Div(
#	[
#	html.H1('Hello Dash', style={'textAlign': 'center', 'color': '#7FDBFF'})
#		dcc.Graph(id = 'live-graph', animate = True),
#		dcc.Interval(
#			id = 'graph-update',
#			interval = 900,
#			n_intervals = 0
#		),
#	]
#)



app_color = {"graph_bg": "#082255", "graph_line": "#007ACE"}
app.layout = html.Div(children=[
    html.H4(children='RSSI: AP'),

#     html.Div(children='''
#         Received signal strength 60 GHz.
#     '''),

    dcc.Graph(
        id='example-graph',
        animate=True), dcc.Interval(id='graph-update',interval=1000,n_intervals=0),
    html.H4(children='Signal Quality: AP'),

#     html.Div(children='''
#         TX/RX direction of AP.
#     '''),

    dcc.Graph(
        id='example-graph1',
        animate=True), dcc.Interval(id='graph-update1',interval=1010,n_intervals=0),
],className="column",)

@app.callback(
    Output('example-graph', 'figure'),
    [ Input('graph-update', 'n_intervals') ]
)

def update_graph_scatter(n):
    X.append(X[-1]+1)
    if len(Y)==19:
        del Y[0]
    Y.append(rssi()[0])

    data = plotly.graph_objs.Scatter(
            x=list(X),
            y=list(Y),

            name='Scatter',
            mode= 'lines+markers'
    )
    line=dict(color="red", width=2),
    layout = dict(
    #plot_bgcolor=app_color["graph_bg"],
    #paper_bgcolor=app_color["graph_bg"],
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#000000"
    ),
    height=500,
    width =700,
    #="Time (S)",
    title="Received Signal Strength",
    xaxis={
        "range": [min(X),max(X)],
        "showline": True,
        "zeroline": False,
        "fixedrange": True,
        "title": "Time (S)",
    },
    yaxis={
        "range": [-65,-48],
        "showgrid": True,
        "showline": True,
        "fixedrange": True,
        "zeroline": False,
       # "gridcolor": app_color["graph_line"],
        "nticks": 6,
        "title": 'RSS (dBm)'
    },)

    return {'data': [data],
            'layout' : layout}

@app.callback(
    Output('example-graph1', 'figure'),
    [ Input('graph-update1', 'n_intervals') ]
)

def update_graph_scatter(n):
    X1.append(X1[-1]+1)
    if len(Y1)==19:
        del Y1[0]
    Y1.append(txsector1)

    data = plotly.graph_objs.Scatter(
            x=list(X1),
            y=list(Y1),

            name='Scatter',
            mode= 'lines+markers'
    )

    return {'data': [data],
            'layout' : go.Layout(title="Signal Quality",xaxis=dict(range=[min(X1),max(X1)]),yaxis = dict(range = [0,100],nticks=10), xaxis_title="Time (S)",
    yaxis_title="Signal (%)", font=dict(
        family="Courier New, monospace",
        size=20,
        color="#000000"
    ))}


if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1344, in dispatch
    cb = self.callback_map[output]
KeyError: 'wind-speed3.figure'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1502, in dispatch_reque

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1344, in dispatch
    cb = self.callback_map[output]
KeyError: 'wind-speed1.figure'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1382, in dispatch
    rais

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1344, in dispatch
    cb = self.callback_map[output]
KeyError: 'wind-speed2.figure'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1382, in dispatch
    rais

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1344, in dispatch
    cb = self.callback_map[output]
KeyError: 'wind-speed3.figure'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1382, in dispatch
    rais

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1344, in dispatch
    cb = self.callback_map[output]
KeyError: 'wind-speed1.figure'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1382, in dispatch
    rais

...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1344, in dispatch
    cb = self.callback_map[output]
KeyError: 'wind-speed3.figure'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1382, in dispatch
    rais

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1344, in dispatch
    cb = self.callback_map[output]
KeyError: 'wind-speed3.figure'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1382, in dispatch
    rais

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1344, in dispatch
    cb = self.callback_map[output]
KeyError: 'wind-speed2.figure'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1382, in dispatch
    rais

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1344, in dispatch
    cb = self.callback_map[output]
KeyError: 'wind-speed1.figure'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1382, in dispatch
    rais

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1344, in dispatch
    cb = self.callback_map[output]
KeyError: 'wind-speed1.figure'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/usr/local/lib/python3.6/dist-packages/dash/dash.py", line 1382, in dispatch
    rais